In [1]:
import numpy as np
import pandas as pd
import random as rd
import array
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

c:\users\arina\appdata\local\programs\python\python37-32\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\arina\appdata\local\programs\python\python37-32\lib\site-packages\numpy\.libs\libopenblas.2V74HQ3MKNZHDCKJELIPPY7V6QMK3UOZ.gfortran-win32.dll
c:\users\arina\appdata\local\programs\python\python37-32\lib\site-packages\numpy\.libs\libopenblas.SVHFG5YE3RK3Z27NVFUDAPL2O3W6IMXW.gfortran-win32.dll
  stacklevel=1)


In [2]:
dataTable = pd.read_csv('cardio_train.csv', delimiter=';').drop('id', 1)

In [3]:
def ageInYears(dataTable):
    ageInYears = []
    for i in range(len(dataTable)):
        ageInYears.append(int(round(dataTable.iloc[i]['age']/365)))
    return ageInYears

In [4]:
ageInYears = ageInYears(dataTable)
dataTable['age_years'] = ageInYears
cols = dataTable.columns.tolist()
cols.insert(0, cols.pop(cols.index('age_years')))
dataTable = dataTable.reindex(columns= cols)
del dataTable['age']
dataTable.columns = ['Age_years', 'Gender', 'Height', 'Weight', 'Systolic_bp', 'Diastolic_bp', 'Cholesterol', 'Glucose', 'Smoking', 'Alcohol_intake', 'Physical_activity', 'Cardio_disease' ]

In [5]:
atributes = dataTable.values[:,:-1]
classes = dataTable.values[:,-1]
xTrain, xTest, yTrain, yTest = train_test_split(atributes, classes, test_size = 0.25)

In [6]:
def normalization(matrix, vector):
    #Mean calculation
    mVector = np.mean(vector, axis=0)
    mMatrix = np.mean(matrix, axis=0)
    
    #Standard deviation
    stdVector = np.std(vector, axis = 0)
    stdMatrix = np.std(matrix, axis = 0)
     
    #Normalization of data
    vector = (vector - mVector)/stdVector
    matrix = (matrix - mMatrix)/stdMatrix
    
    return matrix, vector

In [7]:
def sigmoide(z):
    return 1 / (1 + np.exp(-z))

In [8]:
def predict_logistic_regression_binary(matrix_x, w):
    
    predictions = []
    matrix_x = np.insert(matrix_x, 0, 1, axis =1)
    for i in range(matrix_x.shape[0]):
        predictions.append(round(sigmoide(matrix_x[i].dot(w))))
        
    return predictions

In [9]:
def stochastic_gradient_descent(matrix_x, vector_y, epochs, alpha):
    matrix_x = np.insert(matrix_x, 0, 1, axis =1)
    w_matrix = np.ones((matrix_x.shape[1],), dtype=float)
    y_predicted = matrix_x.dot(w_matrix)
    data =np.append(matrix_x, np.split(vector_y, matrix_x.shape[0], axis=0), axis=1)
    mse = []

    for epoch in range(epochs):
        erro = 0
        somatorio = 0
        for i in range(matrix_x.shape[0]):
            w_matrix  = w_matrix + alpha * (vector_y[i] - sigmoide(y_predicted[i])) * matrix_x[i]
            somatorio += (vector_y[i] - sigmoide(y_predicted[i])) * matrix_x[i]
        
        mse.append((-1/matrix_x.shape[0]) * somatorio)        
        data = np.random.permutation(data)
        matrix_x = data[: ,0:matrix_x.shape[1]]
        vector_y = data[:,matrix_x.shape[1]]
        
        y_predicted = matrix_x.dot(w_matrix)  
    
    return w_matrix, mse

In [10]:
def accuracy(predicted, actual):
    total = 0;
    for a,p in zip(actual, predicted):
        if (a == p):
            total +=1;
    return (total/len(actual))*100

In [11]:
matrix_x, vector_y = normalization(xTrain, yTrain)
matrix_x_test, vector_y_test = normalization(xTest, yTest)

In [12]:
grid_search = np.meshgrid(np.array([0.001, 0.0001, 0.00001]), # passo aprendizagem
                          np.array([500, 1000, 1250,1500,2000])) # lambda

for i in range(grid_search[0].shape[0]):
    for j in range(grid_search[1].shape[1]):
        alpha = grid_search[0][i,j]
        lamb = grid_search[1][i,j]
        print("alpha: ",alpha ,"\t","lamb: " ,lamb)
        w, mse = stochastic_gradient_descent(matrix_x, vector_y, lamb, alpha)
        predictions_logistic = predict_logistic_regression_binary(matrix_x_test, w)
        acc = accuracy(predictions_logistic, yTest)
        print('accuracy', acc)

alpha:  0.001 	 lamb:  500


c:\users\arina\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


accuracy 58.47428571428571
alpha:  0.0001 	 lamb:  500
accuracy 58.46857142857142
alpha:  1e-05 	 lamb:  500
accuracy 58.605714285714285
alpha:  0.001 	 lamb:  1000
accuracy 58.47428571428571
alpha:  0.0001 	 lamb:  1000
accuracy 58.46857142857142
alpha:  1e-05 	 lamb:  1000
accuracy 58.55428571428571
alpha:  0.001 	 lamb:  1250
accuracy 58.47428571428571
alpha:  0.0001 	 lamb:  1250
accuracy 58.46857142857142
alpha:  1e-05 	 lamb:  1250
accuracy 58.53142857142857
alpha:  0.001 	 lamb:  1500
accuracy 58.47428571428571
alpha:  0.0001 	 lamb:  1500
accuracy 58.46857142857142
alpha:  1e-05 	 lamb:  1500
accuracy 58.52
alpha:  0.001 	 lamb:  2000
accuracy 58.47428571428571
alpha:  0.0001 	 lamb:  2000
accuracy 58.47428571428571
alpha:  1e-05 	 lamb:  2000
accuracy 58.502857142857145
